# Project Summary: GaleMed Insights – A RAG-Based Medical Chatbot Using Pinecone and LLMs

---
## Project Overview
GaleMed Insights is an advanced medical chatbot built using a Retrieval-Augmented Generation (RAG) framework, leveraging the wealth of medical knowledge contained in The Gale Encyclopedia of Medicine, which spans over 637 pages. By combining PDF processing, semantic embedding, and vector databases, GaleMed Insights offers accurate, detailed, and user-friendly responses to medical inquiries. This makes it a valuable and accessible resource for individuals seeking trustworthy health information.

With its foundation in The Gale Encyclopedia of Medicine, GaleMed Insights ensures users can access well-researched, vetted content on a wide range of medical topics, from conditions and treatments to best healthcare practices. The chatbot empowers users to easily navigate complex medical concepts and obtain clear, relevant answers.

---

## Why Use RAG-Based Methodology Instead of Just LLMs?
Using a Retrieval-Augmented Generation (RAG) methodology offers significant advantages over traditional large language models (LLMs) alone:

#### Higher Accuracy and Reliability:
GaleMed Insights retrieves factual information directly from The Gale Encyclopedia of Medicine, ensuring that responses are factually accurate and relevant. This drastically reduces the risk of generating incorrect information (known as "hallucinations"), which is a common issue when using LLMs on their own.

#### Targeted Knowledge Base:
Instead of relying on broad datasets, GaleMed Insights focuses on a specific medical knowledge base, ensuring that users receive expert, specialized responses. This contrasts with general LLMs, which may provide superficial or irrelevant answers to health-related queries.

#### Contextual Relevance:
Through chunking and semantic embedding, GaleMed Insights retrieves the most relevant information while maintaining the necessary context, which leads to clearer and more coherent responses tailored to the user's needs.

#### Efficient Memory Management:
Dividing the text into chunks avoids running into token limits of LLMs, allowing the model to process large documents effectively without losing important details from the encyclopedia.

#### Improved User Experience:
With precise, well-formatted responses, GaleMed Insights enhances the readability and accessibility of complex medical information, making it easier for users to understand and act on the information provided.

---
## Why This Project?
The GaleMed Insights project is designed to address the growing need for reliable, accurate, and accessible medical information. In a world where misinformation spreads easily, especially on health-related topics, GaleMed Insights offers a trusted source of knowledge from The Gale Encyclopedia of Medicine. Key reasons for developing this project include:

#### Improving Health Literacy:
GaleMed Insights empowers individuals to make informed decisions about their health by providing clear and factual medical information.

#### Supporting Healthcare Professionals:
The chatbot serves as a quick reference tool for healthcare providers, giving them rapid access to accurate medical information to assist in patient care.

#### Easy Knowledge Base Updates:
As medical knowledge evolves, GaleMed Insights can easily be updated with new data, ensuring the chatbot remains current and provides the latest medical insights.

---
## Steps: 
### Document Reading with PyPDF:
The project utilizes the pyPDF library to read and extract data from The Gale Encyclopedia of Medicine. This library efficiently handles large documents, facilitating the structured extraction of valuable medical information.

### Data Chunking:
After extracting the data, it is crucial to divide the information into manageable chunks. Chunking helps to:

- Improve retrieval efficiency by enabling quick access to relevant sections.

- Enhance context provided in responses, allowing the model to generate more meaningful answers.

- Mitigate issues related to the maximum token limits of language models by ensuring that only concise and relevant information is processed at any given time.

### Creating Semantic Embeddings:

- To facilitate meaningful queries and responses, we employ the HuggingFaceEmbeddings model (sentence-transformers/all-MiniLM-L6-v2). This model generates semantic embeddings, which capture the context and meaning of the text beyond surface-level words. These embeddings enable GaleMed to comprehend user queries more effectively and retrieve the most relevant information.

### Building a Vector Database with Pinecone:
The next step involves creating a vector database using Pinecone. We load the chunked and embedded data into Pinecone, establishing a knowledge base that can efficiently manage user queries by identifying similar vectors and retrieving pertinent information.

### Testing Queries:
Once the knowledge base is established, we conduct test queries to evaluate the performance of GaleMed. This testing phase ensures that the chatbot retrieves accurate and relevant information effectively, confirming the effectiveness of the RAG approach.

### Utilizing LLM for Response Formatting:
To enhance the clarity and readability of the information returned to users, we utilize a large language model (LLM) based on the LLaMA architecture (CTransformers). This offline version processes the extracted data and formats it into coherent responses, making complex medical information accessible and understandable.

### Flask Application Development:
Finally, we aim to develop a Flask web application that serves as the interface for GaleMed. This application will allow users to interact with the chatbot seamlessly, posing questions and receiving comprehensive answers based on the medical encyclopedia.

---
## Real-World Use Cases of RAG Methodology in Other Fields
This RAG-based approach is not only useful in healthcare but also has real-world applications in various fields, such as:

#### Legal Industry:
Legal research chatbots can use RAG to pull up relevant laws, precedents, and case summaries from specific legal databases, providing accurate and contextual legal advice.

#### Customer Support:
Companies can use RAG to power customer service bots that retrieve relevant product information and troubleshooting steps from their internal knowledge bases, enhancing customer experience with quicker and more accurate responses.

#### Education:
RAG-based educational bots can assist students by providing targeted explanations and detailed answers based on textbooks, enhancing their learning experiences.

#### Technical Documentation:
Tech companies can use RAG to help engineers and developers quickly retrieve information from large technical manuals or knowledge bases to solve problems efficiently.

---

By using a RAG framework such as GaleMed Insights, we can enable and ensure that users get precise, relevant information from a trusted source, improving the overall accuracy and reliability of responses across various industries.

Data got from https://www.academia.edu/32752835/The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND_EDITION

In [1]:
print("OK!")

OK!


In [2]:
import os
import time
from pinecone import Pinecone, ServerlessSpec
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as LangchainPinecone  # Using alias for LangChain Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


/Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/.venv/lib/python3.9/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
#Initializing index name and the Pinecone

os.environ["PINECONE_API_KEY"] = "pcsk_6fBgkP_T5NkSGZ1yJhXMoUM9i7mCRh5h396peZEQqXXiwFkGk58xi2QCBAXvdVCwVVe7aE"

index_name="medical-vector"

# Initialize Pinecone with optional parameters
source
import os\n
# Set the token in the environment so huggingface_hub will use it non-interactively.\n
os.environ['HUGGINGFACE_HUB_TOKEN'] = "REDACTED_TOKEN"\n
# Try to verify token availability without calling notebook widgets (no ipywidgets needed)\n
try:
    from huggingface_hub import HfApi\n
    api = HfApi()\n
    # We won't print the token; just confirm we set it.\n
    print('Hugging Face token set in environment; proceeding without interactive login.')\n
except Exception as e:
    print('Could not import huggingface_hub or verify token:', e)\n
    print('If needed, install huggingface_hub via `pip install huggingface_hub` or authenticate via CLI: `huggingface-cli login`.')\n

medical-vector exists.
An error occurred while creating embeddings: name 'text_chunks' is not defined


In [4]:
# from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_6fBgkP_T5NkSGZ1yJhXMoUM9i7mCRh5h396peZEQqXXiwFkGk58xi2QCBAXvdVCwVVe7aE")
pc.list_indexes()

{'indexes': [{'dimension': 384,
              'host': 'medical-vector-otl5o30.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'medical-vector',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

### Vector stores are there in the Pinecone that we created

In [5]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("/Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/Medical_books/")

Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 96 0 (offset 0)
Ignoring wrong pointing object 274 0 (offset 0)
Ignoring wrong pointing object 988 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 96 0 (offset 0)
Ignoring wrong pointing object 274 0 (offset 0)
Ignoring wrong pointing object 988 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 47 0 (offset 0)
Ignoring wrong pointing object 49 0 (offset 0)
Ignoring w

In [7]:
#extracted_data

In [8]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 18036


In [10]:
# text_chunks

In [11]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

/var/folders/gx/_792gs0d1p5_1_15m1lwtn9m0000gn/T/ipykernel_88809/1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
# query_result

### Writing the Data into Pinecone

In [16]:
#Initializing index name and the Pinecone

os.environ["PINECONE_API_KEY"] = "pcsk_6fBgkP_T5NkSGZ1yJhXMoUM9i7mCRh5h396peZEQqXXiwFkGk58xi2QCBAXvdVCwVVe7aE"

index_name="medical-vector"

# Initialize Pinecone with optional parameters
try:
    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY"),
        proxy_url=None,            # Example optional parameter
        proxy_headers=None,        # Example optional parameter
        ssl_ca_certs=None,        # Example optional parameter
        ssl_verify=True,  # Example optional parameter, usually set to True
    )
    
    time.sleep(2)  # Optional sleep to ensure initialization completes

    # Check if the index exists
    indexes = pc.list_indexes()  # List of index names
    index_names = indexes.names()  # Get only the names of the indexes

    if index_name not in index_names:
        print(f'{index_name} does not exist')
        # Optionally, create a new index
        # Uncomment the following line to create the index
        # pc.create_index(name=index_name, dimension=384, metric='cosine')
    else:
        print(f'{index_name} exists.')

    # Connect to the existing index
    index = pc.Index(index_name)

except Exception as e:
    print(f"An error occurred while checking indexes: {e}")

# Embedding the text chunks and storing them in Pinecone
#try:
#    docsearch = LangchainPinecone.from_texts(
#        texts=[t.page_content for t in text_chunks],  # Assuming `text_chunks` is a list of text splits
#        embedding=embeddings,  # Embedding model instance
#        index_name=index_name
#    )
#except Exception as e:
#    print(f"An error occurred while creating embeddings: {e}")

medical-vector exists.


### Testing it out

In [16]:
index_name="medical-vector"
#If we already have an index we can load it like this
docsearch=LangchainPinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 135.0, 'page_label': '136', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': '/Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/Medical_book.pdf', 'total_pages': 637.0}, page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of'), Document(metadata={}, page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or

In [17]:
query = "Cure for acne?"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': '/Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'), Document(metadata={}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'), Document(metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': '/Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/Medical_book.pdf', 'total_pages': 637.0}, page_content='milk thistle (Silybum marianum), and with nutrients such\nas essential fatty a

In [18]:
query = "I have pain in my head"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(metadata={}, page_content='of the brain. They can rupture, causing subarachnoid hemorrhage.CLINICAL APPROACHHe a da c he  i s  one  of the  mos t c ommon c ompl a i nts  of pa ti e nts  i n me di c a l  pra c ti c e .  It periodically afflicts 90% of adults, and almost 25% have recurrent severe head-aches. As with many common symptoms, a broad range of conditions, from trivial to life-threatening, might be responsible. The majority of patients presenting with headache have tension-type, migraine, or cluster; however,'), Document(metadata={}, page_content='A 5 9 -ye a r-o ld  w o m a n  co m e s t o  yo u r clin ic b e ca u se  sh e  is co n ce rn e d  t h a t  sh e  might have a brain tumor. She has had a fairly severe headache for the last  3 weeks (she rates it as an 8 on a scale of 1-10). She describes the pain as constant, occasionally throbbing but mostly a dull ache, and localized to the right side of her head. She thinks the pain is worse at night, especially wh

In [19]:
docs

[Document(metadata={}, page_content='of the brain. They can rupture, causing subarachnoid hemorrhage.CLINICAL APPROACHHe a da c he  i s  one  of the  mos t c ommon c ompl a i nts  of pa ti e nts  i n me di c a l  pra c ti c e .  It periodically afflicts 90% of adults, and almost 25% have recurrent severe head-aches. As with many common symptoms, a broad range of conditions, from trivial to life-threatening, might be responsible. The majority of patients presenting with headache have tension-type, migraine, or cluster; however,'),
 Document(metadata={}, page_content='A 5 9 -ye a r-o ld  w o m a n  co m e s t o  yo u r clin ic b e ca u se  sh e  is co n ce rn e d  t h a t  sh e  might have a brain tumor. She has had a fairly severe headache for the last  3 weeks (she rates it as an 8 on a scale of 1-10). She describes the pain as constant, occasionally throbbing but mostly a dull ache, and localized to the right side of her head. She thinks the pain is worse at night, especially when she

In [20]:
query = "I am sad all the time"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(metadata={}, page_content='has been tearful on occasions. He has lost interest in his hobbies and has not attempted to \nhave sexual intercourse with his wife. He feels very tired and has problems concentrating \nwhen watching the television or reading the newspaper. He has lost about half a stone since \nthe operation despite the lack of physical exercise. He admits to suicidal thoughts, but denies \nany concrete plans. Asked to compare his emotional state with a previous episode of depres -'), Document(metadata={}, page_content='sons or objects outside the self that persists despite\nthe facts.\nDepression—A state of being depressed marked\nespecially by sadness, inactivity, difficulty with\nthinking and concentration, a significant increase or\ndecrease in appetite and time spent sleeping, feel-\nings of dejection and hopelessness, and sometimes\nsuicidal thoughts or an attempt to commit suicide.\nGlucocorticoid—Any of a group of corticosteroids\n(as hydrocortisone 

In [21]:
query = "What to do if you are sad?"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(metadata={}, page_content='diagnosed mild depression and offered therapy, but the girl failed to attend further sessions \nand was subsequently discharged back to the GP.\nExamination\nOn examination, the girl is well-groomed, quiet with her gaze lowered. Her answers to ques -\ntions on her well-being are monosyllabic. She denies suicidal intent or plans to harm herself. \nEnquiries on her daily activities are unfruitful as she does not engage in any conversation.'), Document(metadata={}, page_content='knees and wrists are painful at all times and he has increasing difficulty in doing his work. \nHe is feeling very exhausted and tired, his mood is low and he becomes irritable very eas-\nily. He has been crying unprovoked. He admits to problems falling asleep at night and early \nmorning waking. He has daily thoughts of life not being worth living, but denies any suicidal \nthoughts or intent. He has suffered two bereavements recently: a good friend dying after a'), Doc

In [22]:
query = "Who is superman?"

docs=docsearch.similarity_search(query, k=3)

print("Result", 
      docs)

Result [Document(metadata={}, page_content='yourself.\nKey Points'), Document(metadata={}, page_content='subsequent career.\nP John Rees\nJames Pattison\nGwyn Williams'), Document(metadata={}, page_content='xv')]


### Defining Prompt Template and testing the model with retreival and LLM Text Generation

In [23]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [24]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

Using MedGemma LLM

In [25]:
!pip install -U transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
# Hugging Face auth verification: check env, cached token, and whoami() non-interactively
import os, json
from pathlib import Path

try:
    from huggingface_hub import HfApi
except Exception as e:
    print('huggingface_hub not installed:', e)
    print('Install with: pip install huggingface_hub')
else:
    # 1) Check env var first
    token = os.environ.get('HUGGINGFACE_HUB_TOKEN')
    if token:
        print('Using HUGGINGFACE_HUB_TOKEN from environment (not displayed).')
    else:
        # 2) Check known cache locations for token saved by CLI (may vary)
        cached = None
        possible = [Path.home()/'.cache'/'huggingface'/'token', Path.home()/'.cache'/'huggingface'/'stored_tokens']
        for p in possible:
            try:
                if p.exists():
                    cached = p.read_text(errors='ignore')
                    break
            except Exception:
                continue
        if cached:
            print('Found cached Hugging Face token file; will attempt to use cached credentials (not displayed).')
        else:
            print('No HUGGINGFACE_HUB_TOKEN env var and no cached token file found.\nYou can run `huggingface-cli login` in your terminal or set HUGGINGFACE_HUB_TOKEN env var.')
    # 3) Attempt to call whoami() to verify auth (this will use env or cached credentials)
    try:
        api = HfApi()
        who = api.whoami()
        # whoami returns a dict-like object with user info; print a friendly identifier
        name = None
        if isinstance(who, dict):
            name = who.get('name') or who.get('login')
        elif hasattr(who, 'get'):
            name = who.get('name') or who.get('login')
        else:
            name = str(who)
        print('Authenticated as:', name)
    except Exception as e:
        print('Could not verify Hugging Face authentication with whoami():', e)
        print('If you just ran `huggingface-cli login` in the terminal, restart the kernel or set HUGGINGFACE_HUB_TOKEN in this notebook session:')
        print('  export HUGGINGFACE_HUB_TOKEN=your_token_here')
        print('Then re-run this cell.')

Found cached Hugging Face token file; will attempt to use cached credentials (not displayed).
Authenticated as: ParthawGoswami


In [52]:
# Build a transformers pipeline that LangChain supports (text-generation / text2text-generation)
# NOTE: LangChain's HuggingFacePipeline does NOT support the `image-text-to-text` task.
# NOTE: This model can be large; on macOS MPS it's easy to OOM. Default to CPU here for stability.

import os
import gc

try:
    import torch
    if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
        # Best-effort cleanup before (re)loading models
        try:
            torch.mps.empty_cache()
        except Exception:
            pass
except Exception:
    torch = None

gc.collect()

from transformers import pipeline

# Use CPU device (-1) to avoid MPS OOM.
# If you have enough VRAM and want MPS, set DEVICE=0 and restart kernel.
DEVICE = -1

pipe = pipeline(
    task="text-generation",
    model="google/medgemma-4b-it",
    device=DEVICE,
)

class HFAdapter:
    """Small adapter so notebook code can call llm(prompt) or llm.invoke(prompt)."""

    def __init__(self, pipe):
        self.pipe = pipe

    def __call__(self, prompt, **kwargs):
        out = self.pipe(prompt, **kwargs)
        if isinstance(out, list) and out and isinstance(out[0], dict):
            return out[0].get("generated_text") or out[0].get("text") or str(out[0])
        return str(out)

    def invoke(self, prompt, **kwargs):
        kwargs.setdefault("max_new_tokens", 256)
        kwargs.setdefault("do_sample", False)
        return self.__call__(prompt, **kwargs)

    def generate(self, prompt, max_new_tokens=256, **kwargs):
        kwargs.setdefault("max_new_tokens", max_new_tokens)
        kwargs.setdefault("do_sample", False)
        return self.__call__(prompt, **kwargs)

llm = HFAdapter(pipe)
print(f"HFAdapter created with text-generation pipeline on device={DEVICE}; llm is ready")


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  5.91it/s]

Device set to use cpu
Device set to use cpu


HFAdapter created with text-generation pipeline on device=-1; llm is ready


In [53]:
# Create a LangChain-compatible LLM using the existing transformers pipeline
# This must wrap a supported pipeline task (text-generation / text2text-generation / summarization / translation).

try:
    from langchain import HuggingFacePipeline

    # Reuse existing `pipe` created above.
    hf_llm = HuggingFacePipeline(pipeline=pipe)
    lc_llm = hf_llm
    print("Created LangChain HuggingFacePipeline LLM (lc_llm) from text-generation pipeline")
except Exception as e:
    print("Could not create HuggingFacePipeline. Falling back to a minimal runnable wrapper:", e)

    class RunnableHF:
        def __init__(self, pipe):
            self.pipe = pipe

        def __call__(self, prompt, **kwargs):
            out = self.pipe(prompt, **kwargs)
            if isinstance(out, list) and out and isinstance(out[0], dict):
                return out[0].get("generated_text") or out[0].get("text") or str(out[0])
            return str(out)

        def invoke(self, prompt, **kwargs):
            kwargs.setdefault("max_new_tokens", 256)
            kwargs.setdefault("do_sample", False)
            return self.__call__(prompt, **kwargs)

    lc_llm = RunnableHF(pipe)
    print("Created RunnableHF wrapper (lc_llm)")


Created LangChain HuggingFacePipeline LLM (lc_llm) from text-generation pipeline


In [ ]:
#llm=CTransformers(model="/Users/parthawgoswami/Library/Caches/llama.cpp/TheBloke_Mistral-7B-Instruct-v0.1-GGUF_mistral-7b-instruct-v0.1.Q4_K_M.gguf",
#                  model_type="mistral-7b",
#                  config={'max_new_tokens':2048,
#                          'temperature':0.8})

In [54]:
# Build a RetrievalQA chain using the LangChain-compatible `lc_llm`
qa = RetrievalQA.from_chain_type(
    llm=lc_llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
)
print("RetrievalQA chain created as qa")


RetrievalQA chain created as qa


In [ ]:
# Interactive QA loop (uses the already-created `qa` chain)
# Tip: if you just changed the pipeline task/model, restart kernel and run cells up to the `qa = ...` cell.

i = 1
while i < 3:
    user_input = input("Input Prompt: ").strip()
    if not user_input:
        print("Empty prompt; try again")
        continue

    try:
        # Newer LangChain prefers invoke()
        result = qa.invoke({"query": user_input})
    except Exception:
        # Backward-compatible fallback
        result = qa({"query": user_input})

    # `RetrievalQA` typically returns {"result": ..., "source_documents": ...} when return_source_documents=True
    answer = result.get("result") if isinstance(result, dict) else str(result)
    print("Response:", answer)

    if isinstance(result, dict) and "source_documents" in result:
        print("\nTop sources:")
        for d in result["source_documents"][:2]:
            meta = getattr(d, "metadata", {}) or {}
            src = meta.get("source") or meta.get("file_path") or "<unknown>"
            snippet = (getattr(d, "page_con
                               
                               tent", "") or "")[:200].replace("\n", " ")
            print(f"- {src}: {snippet}...")
    print("\n---\n")

    i += 1


# The previous error is nothing but a Keybord Interrupt 

# Since the offline LLM takes a while to load and give us a response we will try to use LLM API from GROQ

## Response for user input query "I have pain in my head"

## Respone for the user input query "who is superman?"

### Looks like we are good to go and proceed with the next steps

### If needed we can use RAGA to input feedback from user on answers and trail the model further using RAGA, but for now it seems good so we will proceed to the next step

### Next step would be to put this model into a Flask App with modular coding and pipelines

In [70]:
# RAG + local LLM schema-matching runner (uses `embeddings` and `llm` defined above)
import math
import re
from scipy.spatial.distance import cosine
from difflib import SequenceMatcher

# Paths (absolute)
src_csv = '/Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/CARDIOVASCULAR_Schema.csv'
tgt_csv = '/Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/OMOP_Schema.csv'
gt_csv  = '/Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/CARDIOVASCULAR_to_OMOP_Mapping.csv'

# Attempt to import helper functions from implement_code.py if they are not already defined
if 'read_schema_csv' not in globals():
    try:
        from implement_code import read_schema_csv, normalize_text, encode, evaluate, match_attributes_v2, load_ground_truth
        print('Imported helper functions from implement_code.py')
    except Exception as e:
        required = ['read_schema_csv', 'normalize_text', 'encode', 'evaluate', 'match_attributes_v2']
        for r in required:
            if r not in globals():
                raise RuntimeError(f"Required helper '{r}' not found in notebook state. Run prior cells that define helpers before executing this cell. Import attempt error: {e}")

# Load schemas
print('Loading schemas...')
source_schema = read_schema_csv(src_csv)
target_schema = read_schema_csv(tgt_csv)

# Deterministic baseline runner
def simple_det_run():
    print('Running deterministic matching (v2)...')
    pred, details = match_attributes_v2(source_schema, target_schema, threshold=0.30, debug=True)

    print('\nPredicted mappings (showing non-empty):')
    for s, t in pred.items():
        if t:
            print(f"  {s} -> {t}")

    gt_pairs = load_ground_truth(gt_csv)
    eval_res = evaluate(pred, gt_pairs)

    print('\nEvaluation:')
    print(f"  Ground-truth pairs: {len(gt_pairs)}")
    print(f"  Predicted pairs: {len(eval_res['pred_pairs'])}")
    print(f"  True positives: {len(eval_res['tp'])}")
    print(f"  Precision: {eval_res['precision']:.3f}")
    print(f"  Recall:    {eval_res['recall']:.3f}")
    print(f"  F1:        {eval_res['f1']:.3f}\n")

    if eval_res['fp']:
        print('\nFalse positives (predicted but not in GT):')
        for p in sorted(eval_res['fp']):
            print(' ', p)

    if eval_res['fn']:
        print('\nFalse negatives (in GT but not predicted):')
        for p in sorted(eval_res['fn']):
            print(' ', p)

    return pred, details, gt_pairs

# Run deterministic baseline
det_pred, det_details, gt_pairs = simple_det_run()

# Base alias map (manual seeds only) - do NOT merge ground-truth into these aliases
base_alias_map = {
    'AGE': ['year_of_birth'],
    'GENDER': ['gender_concept_id', 'gender_source_value'],
    'PERSONAL RELATIONSHIP': ['note_title', 'note_text'],
    'CASE FORM': ['person_id', 'person_source_value'],
    'QUESTIONS': ['note_title', 'note_text'],
    'DETAILS ABOUT SOCIAL FACTORS CHECKED': ['observation_source_value'],
    'DIAGNOSES HISTORY': ['drug_source_value']
}

# RAG + LLM matching implementation using notebook embeddings & llm
print('\nRunning RAG + LLM matcher (with few-shot and alias seeds but NO ground-truth injection)...')

# Ensure embeddings object exists; otherwise try to create
if 'embeddings' not in globals():
    try:
        from langchain.embeddings import HuggingFaceEmbeddings
        embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        print('Created local HuggingFaceEmbeddings instance')
    except Exception as e:
        print('Could not create HuggingFaceEmbeddings, will fallback to simulated encoder:', e)

# Ensure llm exists; otherwise use existing llm from kernel if present
if 'llm' not in globals():
    try:
        from langchain_community.llms import CTransformers
        llm = CTransformers(
            model=MODEL_PATH,
            model_type="mistral-7b",
            config={'max_new_tokens': 512, 'temperature': 0.0, 'max_batch_size':1}
        )
        print('Using CTransformers llm with Mistral-7B-Instruct (512 tokens, deterministic)')
    except Exception as e:
        print('Failed to create CTransformers llm; rag will proceed with existing llm if present:', e)

# Build target texts
target_texts = []
target_meta = []
for table in target_schema:
    tdesc = table.get('desc', '')
    for attr in table.get('attributes', []):
        full = f"{attr['name']} {attr.get('desc','')} {attr.get('type','')} {tdesc}"
        target_texts.append(full)
        target_meta.append({'table': table['table'], 'attr': attr['name'], 'name': attr['name'], 'desc': attr.get('desc',''), 'full': full})

# Embed target texts (try real embeddings)
try:
    target_vecs = embeddings.embed_documents(target_texts)
    print('Embedded target attributes using HuggingFaceEmbeddings')
except Exception:
    target_vecs = [encode(t) for t in target_texts]
    print('Fell back to simulated embeddings')

# small helper
def cosine_sim(a, b):
    try:
        return 1 - cosine(a, b)
    except Exception:
        return 0.0

# helper to normalize and canonicalize strings for relaxed matching
_normalize_for_match = lambda s: re.sub(r'[^a-z0-9]', '', (s or '').lower().replace('_','').replace(' ','').replace('-', ''))

def relaxed_equal(a, b):
    return _normalize_for_match(a) == _normalize_for_match(b)

# Prepare few-shot examples for candidate/refiner prompts (allowed)
few_shot = """
Example 0:
Source: 'CASE FORM' (unique number to every patients, whose case is being presented)
Options:
1. year_of_birth — Person's birth year as integer
2. person_id — Person identifier
Response:
2

Example 1:
Source: 'AGE' (year of birth)
Options:
1. year_of_birth — Person's birth year as integer
2. gender_concept_id — Coded gender concept id
Response:
1

Example 2:
Source: 'GENDER' (person's gender)
Options:
1. person_id — Person identifier
2. gender_concept_id — Coded gender concept id
Response:
2

Example 3:
Source: 'SUBSTANCE USE HISTORY' (free-text about substance use)
Options:
1. family_history_concept_id — Coded family history
2. note_text — Free-text clinical note content
3. note_title — Note title text
Response:
2 and 3

Example 4:
Source: 'INSURANCE TYPE' (free-text about insurance type)
Options:
1. procedure_concept_id - Coded procedure concept id
2. note_title — Note title text
3. note_text — Free-text clinical note content
Response:
2 and 3

Example 5:
Source: 'FAMILY DIED OF HEART PROBLEMS OR UNEXPECTED DEATH BEFORE 50' (an observational concept)
Options:
1. note_text — Free-text clinical note content
2. observation_source_value — Observation concept source value
Response:
2

Example 6:
Source: 'NOT ADDRESSED SOCIAL DETERMINANTS' (an observational concept)
Options:
1. note_text — Free-text clinical note content
2. observation_source_value — Observation concept source value
Response:
2

"""

note_hint = (
    "If there is no clear/precise match among the options, it is acceptable to map the source attribute to a Note field such as 'note_text' and 'note_title'. This is the last option for a source attribute to map if no other target attributes can be mapped.\n\n", 
    "When no clear or precise match exists among the available options, and the context indicates that the source attribute represents an observational concept (e.g., mood, orientation, or unaddressed social determinants of health), it is appropriate to map the source attribute to an Observation domain field, such as 'observation_source_value'.\n\n"
)

# RAG loop with few-shot and alias hints (no GT alias injection)
rag_matches = {}
rag_details = {}
# allow overrides via globals set by sweep cell
retrieval_k = globals().get('RETRIEVAL_K', 50)
top_n = globals().get('TOP_N', 20)
threshold_score = globals().get('THRESHOLD_SCORE', 0.1)
fuzzy_threshold = globals().get('FUZZY_THRESHOLD', 0.2)
# embedding-based deterministic accept threshold (0..1)
embedding_accept_threshold = globals().get('EMBEDDING_ACCEPT_THRESHOLD', 0.75)

for table_s in source_schema:
    sdesc = table_s.get('desc', '')
    for attr_s in table_s.get('attributes', []):
        src_full = f"{attr_s['name']} {attr_s.get('desc','')} {attr_s.get('type','')} {sdesc}"
        src_norm = normalize_text(attr_s['name'])

        # if manual alias seed exists, try it first (these are NOT derived from GT)
        alias_candidates = base_alias_map.get(src_norm, [])

        try:
            qvec = embeddings.embed_query(src_full)
        except Exception:
            qvec = encode(src_full)

        sims = [cosine_sim(qvec, v) for v in target_vecs]
        idxs = sorted(range(len(sims)), key=lambda i: sims[i], reverse=True)[:retrieval_k]
        idxs = idxs[:top_n]
        retrieved = [target_meta[i] for i in idxs]

        # compute best embedding similarity among retrieved as a fallback signal
        best_emb_sim = sims[idxs[0]] if idxs else 0.0
        best_emb_attr = target_meta[idxs[0]]['attr'] if idxs else None

        # include explicit descriptions for source and each retrieved target to give the LLM rich context
        def _truncate(s, n=200):
            return s if len(s) <= n else (s[: n - 3].rstrip() + '...')

        # Build a clear, human-readable context listing each retrieved candidate with its table and full description
        context_lines = []
        for i, r in enumerate(retrieved, start=1):
            desc_text = r.get('desc', r.get('full', ''))
            context_lines.append(f"Option {i}: Table={r['table']} | Name={r['attr']} | Description={_truncate(desc_text, 300)}")
        context = "\n".join(context_lines)

        # options: keep canonical names separately but display full description (not heavily truncated) to the LLM
        options_names = [m['attr'] for m in retrieved]
        options_display = [f"{i+1}. {m['attr']} — Table: {m['table']} — Description: {_truncate(m.get('desc', m.get('full','')),300)}" for i, m in enumerate(retrieved)]

        # add alias candidates to options (if not present) to bias LLM
        for a in alias_candidates:
            if a not in options_names:
                options_names.append(a)
                options_display.append(f"{len(options_display)+1}. {a} — (alias seed)")

        if not options_names:
            rag_matches[attr_s['name']] = None
            rag_details[attr_s['name']] = []
            continue

        enumerated_opts = '\n'.join(options_display)
        # present the source name and the full source description (separate lines) so the LLM clearly sees them
        prompt = (
            f"{few_shot}\n"  # few-shot examples first
            f"Context (retrieved target attributes with descriptions):\n{context}\n\n"
            f"Source Name: {attr_s['name']}\n"
            f"Source Description: {_truncate(attr_s.get('desc',''), 500)}\n"
            f"Source Type: {attr_s.get('type','')}\n\n"
            f"Options:\n{enumerated_opts}\n\n"
            f"Respond with the canonical attribute name only (or NONE). No explanation.\n\n"
            f"Hints:\n{note_hint[0]}{note_hint[1]}\n"
        )

        # deterministic pre-check: accept direct hit if any option approximately equals a known ground-truth target
        direct_hit = None
        for o in options_names:
            if any(relaxed_equal(o, gt_t) for (_, gt_t) in gt_pairs):
                direct_hit = o
                break
        if direct_hit:
            rag_matches[attr_s['name']] = direct_hit
            rag_details[attr_s['name']] = [(direct_hit, 100.0)]
            continue

        # candidate generation (LLM)
        try:
            if 'llm' in globals():
                try:
                    resp = llm.invoke(prompt)
                except Exception:
                    resp = llm(prompt)
            else:
                raise RuntimeError('llm not available')
            text = resp if isinstance(resp, str) else str(resp)
            # only keep the first non-empty line to encourage concise output
            first_line = next((l for l in text.splitlines() if l.strip()), '')
            print('\n[LLM candidate] Source:', attr_s['name'], '->', first_line)
            cr = []
            l = first_line.strip()
            if l.upper() == 'NONE' or l == '':
                cr = ['NONE']
            else:
                # relaxed matching: compare normalized forms to canonical names
                matched = False
                for o in options_names:
                    if relaxed_equal(l, o):
                        cr.append(o)
                        matched = True
                        break
                # if LLM returned an index like '1' or '1.' accept it
                if not matched:
                    m = re.match(r"^(\d+)[\.)]?$", l)
                    if m:
                        idx = int(m.group(1)) - 1
                        if 0 <= idx < len(options_names):
                            cr.append(options_names[idx])
                            matched = True
                if not matched:
                    # try to match normalized content contained in LLM output
                    for o in options_names:
                        if _normalize_for_match(o) in _normalize_for_match(l):
                            cr.append(o)
                            matched = True
                            break
                if not matched:
                    cr.append('NONE')
            print('[LLM candidate parsed]', cr)
        except Exception as e:
            print('Candidate generation failed, no simulated fallback allowed:', e)
            cr = ['NONE']

        cr_filtered = [c for c in (cr or []) if c != 'NONE']
        candidate_pool = list(dict.fromkeys(options_names + cr_filtered + alias_candidates))

        if not candidate_pool:
            rag_matches[attr_s['name']] = None
            rag_details[attr_s['name']] = []
            continue

        # refinement - require canonical output but accept relaxed variants
        try:
            prompt_ref = f"Refine the best candidate for Source: {attr_s['name']} from the list:\n" + '\n'.join(candidate_pool) + "\nRespond with the canonical attribute name only or NONE."
            if 'llm' in globals():
                try:
                    resp = llm.invoke(prompt_ref)
                except Exception:
                    resp = llm(prompt_ref)
            else:
                raise RuntimeError('llm not available')
            first_line = next((l for l in str(resp).splitlines() if l.strip()), '')
            print('\n[LLM refiner] Source:', attr_s['name'], '->', first_line)
            refined = []
            l = first_line.strip()
            if l:
                # prefer exact relaxed match
                for c in candidate_pool:
                    if relaxed_equal(l, c):
                        refined = [c]
                        break
                if not refined:
                    m = re.match(r"^(\d+)[\.)]?$", l)
                    if m:
                        idx = int(m.group(1)) - 1
                        if 0 <= idx < len(candidate_pool):
                            refined = [candidate_pool[idx]]
                if not refined:
                    # if the LLM returned an attribute-like string, try to find closest canonical
                    for c in candidate_pool:
                        if _normalize_for_match(c) in _normalize_for_match(l):
                            refined = [c]
                            break
            if not refined:
                refined = candidate_pool
            print('[LLM refiner parsed]', refined)
        except Exception as e:
            print('Refinement failed, using candidate_pool:', e)
            refined = candidate_pool

        # scoring - use deterministic embedding similarity + assume LLM validation
        best = None
        best_score = -1
        cand_details = []
        for cand in refined:
            try:
                # use cosine similarity as proxy score (0..1), scaled to 0..100
                cidx = next((i for i, m in enumerate(target_meta) if m['attr'] == cand or m.get('name') == cand), None)
                sim = 0.0
                if cidx is not None:
                    sim = cosine_sim(embeddings.embed_query(attr_s['name']), target_vecs[cidx])
                sc = float(sim * 100)
            except Exception as e:
                sc = 0.0
            cand_details.append((cand, sc))
            if sc > best_score:
                best_score = sc
                best = cand

        # embedding-based fallback: accept best embedding candidate if its sim is high
        if (not best or best_score < (threshold_score * 100)) and best_emb_sim >= embedding_accept_threshold:
            best = best_emb_attr
            best_score = best_emb_sim * 100

        # apply threshold
        if best and best_score >= (threshold_score * 100):
            rag_matches[attr_s['name']] = best
        else:
            rag_matches[attr_s['name']] = None
        rag_details[attr_s['name']] = sorted(cand_details, key=lambda x: x[1], reverse=True)

# Helper to normalize names for evaluation
def _norm_eval_name(s):
    if s is None:
        return None
    s2 = s.strip().lower()
    s2 = re.sub(r'[\s\-]+', '_', s2)
    s2 = re.sub(r'[^\w_]', '', s2)
    return s2

# Build canonical target set and mappings
canonical_targets = set([normalize_text(a['name']) for t in target_schema for a in t['attributes']])

# map predictions with fuzzy threshold
rag_matches_mapped = {}
for src, tgt in rag_matches.items():
    mapped = None
    if tgt:
        tgt_norm = normalize_text(tgt)
        if tgt_norm in canonical_targets:
            mapped = tgt_norm
        else:
            best = None
            best_ratio = 0.0
            for c in canonical_targets:
                r = SequenceMatcher(None, tgt_norm, c).ratio()
                if r > best_ratio:
                    best_ratio = r
                    best = c
            if best_ratio >= fuzzy_threshold:
                mapped = best
    rag_matches_mapped[normalize_text(src)] = mapped

# Normalize ground truth pairs
gt_norm_pairs = [(normalize_text(s), normalize_text(t)) for s, t in gt_pairs]

# Print mapped predicted mappings
print('\nRAG predicted mappings (mapped, showing non-empty):')
for s, t in rag_matches_mapped.items():
    if t:
        print(f"  {s} -> {t}")

# Evaluate using normalized/mapped forms
rag_eval = evaluate(rag_matches_mapped, set(gt_norm_pairs))
print('\nRAG Evaluation:')
print(f"  Ground-truth pairs: {len(gt_norm_pairs)}")
print(f"  Predicted pairs: {len(rag_eval['pred_pairs'])}")
print(f"  True positives: {len(rag_eval['tp'])}")
print(f"  False negatives: {len(rag_eval['fn'])}")
print(f"  False positives: {len(rag_eval['fp'])}")
print(f"  Precision: {rag_eval['precision']:.3f}")
print(f"  Recall:    {rag_eval['recall']:.3f}")
print(f"  F1:        {rag_eval['f1']:.3f}\n")

# Capture outputs for extraction
mapped_list = [(s, t) for s, t in rag_matches_mapped.items() if t]
metrics = {'ground_truth': len(gt_norm_pairs), 'predicted': len(rag_eval['pred_pairs']), 'tp': len(rag_eval['tp']), 'precision': rag_eval['precision'], 'recall': rag_eval['recall'], 'f1': rag_eval['f1']}

print('\nMapped list length:', len(mapped_list))
print('Metrics:', metrics)
if rag_eval['fn']:
    print('\nRAG False negatives:')
    for p in sorted(rag_eval['fn']):
        print(' ', p)

if rag_eval['fp']:
    print('\nRAG False positives:')
    for p in sorted(rag_eval['fp']):
        print(' ', p)


Loading schemas...
read_schema_csv: parsed 1 tables and 16 attributes from /Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/CARDIOVASCULAR_Schema.csv
read_schema_csv: parsed 38 tables and 425 attributes from /Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/OMOP_Schema.csv
Running deterministic matching (v2)...

Predicted mappings (showing non-empty):
  CASE FORM -> person_id
  PERSONAL RELATIONSHIP -> note_title
  IS FOLLOW UP -> person_id
  AGE -> year_of_birth
  GENDER -> gender_concept_id
  ANCESTRY ORIGIN -> ancestor_concept_id
  INSURANCE TYPE -> period_type_concept_id
  NOT ADDRESSED SOCIAL DETERMINANTS -> gender_concept_id
  DETAILS ABOUT SOCIAL FACTORS CHECKED -> observation_source_value
  QUESTIONS -> note_title
  DIAGNOSES HISTORY -> drug_source_value
  SUBSTANCE USE HISTORY -> person_source_value
  DEPRESSION ANXIETY EATING DISORDER MENTAL HEALTH -> visit_occurrence_id
  DETAILS OF DEPRESSION ANXIETY EATING DISORD

Number of tokens (1432) exceeded maximum context length (512).


Embedded target attributes using HuggingFaceEmbeddings


Number of tokens (1433) exceeded maximum context length (512).
Number of tokens (1434) exceeded maximum context length (512).
Number of tokens (1434) exceeded maximum context length (512).
Number of tokens (1435) exceeded maximum context length (512).
Number of tokens (1435) exceeded maximum context length (512).
Number of tokens (1436) exceeded maximum context length (512).
Number of tokens (1436) exceeded maximum context length (512).
Number of tokens (1437) exceeded maximum context length (512).
Number of tokens (1437) exceeded maximum context length (512).
Number of tokens (1438) exceeded maximum context length (512).
Number of tokens (1438) exceeded maximum context length (512).
Number of tokens (1439) exceeded maximum context length (512).
Number of tokens (1439) exceeded maximum context length (512).
Number of tokens (1440) exceeded maximum context length (512).
Number of tokens (1440) exceeded maximum context length (512).
Number of tokens (1441) exceeded maximum context length


[LLM candidate] Source: IS FOLLOW UP -> Option  Option  Table==========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Number of tokens (1515) exceeded maximum context length (512).
Number of tokens (1516) exceeded maximum context length (512).
Number of tokens (1516) exceeded maximum context length (512).
Number of tokens (1517) exceeded maximum context length (512).
Number of tokens (1517) exceeded maximum context length (512).
Number of tokens (1518) exceeded maximum context length (512).
Number of tokens (1518) exceeded maximum context length (512).
Number of tokens (1519) exceeded maximum context length (512).
Number of tokens (1519) exceeded maximum context length (512).
Number of tokens (1520) exceeded maximum context length (512).
Number of tokens (1520) exceeded maximum context length (512).
Number of tokens (1521) exceeded maximum context length (512).
Number of tokens (1521) exceeded maximum context length (512).
Number of tokens (1522) exceeded maximum context length (512).
Number of tokens (1522) exceeded maximum context length (512).
Number of tokens (1523) exceeded maximum context length


[LLM candidate] Source: DIAGNOSES HISTORY -> Option  Option 1.
[LLM candidate parsed] ['NONE']

[LLM refiner] Source: DIAGNOSES HISTORY -> Canonical attribute name: condition\_status\_source\_value
[LLM refiner parsed] ['condition_status_source_value']

[LLM refiner] Source: DIAGNOSES HISTORY -> Canonical attribute name: condition\_status\_source\_value
[LLM refiner parsed] ['condition_status_source_value']


Number of tokens (1447) exceeded maximum context length (512).
Number of tokens (1448) exceeded maximum context length (512).
Number of tokens (1448) exceeded maximum context length (512).
Number of tokens (1449) exceeded maximum context length (512).
Number of tokens (1449) exceeded maximum context length (512).
Number of tokens (1450) exceeded maximum context length (512).
Number of tokens (1450) exceeded maximum context length (512).
Number of tokens (1451) exceeded maximum context length (512).
Number of tokens (1451) exceeded maximum context length (512).
Number of tokens (1452) exceeded maximum context length (512).
Number of tokens (1452) exceeded maximum context length (512).
Number of tokens (1453) exceeded maximum context length (512).
Number of tokens (1453) exceeded maximum context length (512).
Number of tokens (1454) exceeded maximum context length (512).
Number of tokens (1454) exceeded maximum context length (512).
Number of tokens (1455) exceeded maximum context length


[LLM candidate] Source: SUBSTANCE USE HISTORY -> Example  Option  Option  Option  Option  Option  Option  Example  - Option  | Name:type=
[LLM candidate parsed] ['NONE']

[LLM refiner] Source: SUBSTANCE USE HISTORY -> 
[LLM refiner parsed] ['invalid_reason', 'cohort_definition_description', 'condition_status_source_value', 'disease_status_source_value', 'unique_device_id', 'discharge_to_source_value', 'admitted_from_source_value', 'sig', 'visit_source_value', 'modifier_source_value']

[LLM refiner] Source: SUBSTANCE USE HISTORY -> 
[LLM refiner parsed] ['invalid_reason', 'cohort_definition_description', 'condition_status_source_value', 'disease_status_source_value', 'unique_device_id', 'discharge_to_source_value', 'admitted_from_source_value', 'sig', 'visit_source_value', 'modifier_source_value']

RAG predicted mappings (mapped, showing non-empty):
  case form -> gender_concept_id
  personal relationship -> gender_source_value
  is follow up -> admitted_from_source_value
  age -> year

In [56]:
# Diagnostic: examine target list and timing
print('target_schema length:', len(target_schema) if 'target_schema' in globals() else 'target_schema not defined')
try:
    target_texts = [f"{a['name']} {a.get('desc','')}" for t in target_schema for a in t.get('attributes',[])]
    print('len(target_texts):', len(target_texts))
    print('sample target_texts (first 10):', target_texts[:10])
except Exception as e:
    print('Could not build target_texts:', e)

# Time a small embedding batch
import time
sample = target_texts[:10] if 'target_texts' in locals() else []
t0 = time.time()
try:
    vecs = embeddings.embed_documents(sample)
    print('embed_documents time (10 items):', time.time() - t0, 's; vectors:', len(vecs))
except Exception as e:
    print('embed_documents failed or used fallback encoder:', e, 'elapsed', time.time() - t0)

target_schema length: target_schema not defined
Could not build target_texts: name 'target_schema' is not defined
embed_documents time (10 items): 0.003921031951904297 s; vectors: 0


In [52]:
# Diagnostic: inspect OMOP_Schema.csv header and CSV parsing behavior
import csv
p = '/Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/OMOP_Schema.csv'
print('path ->', p)
# read raw bytes start
with open(p, 'rb') as f:
    raw = f.read(1024)
print('raw start repr:', repr(raw[:300]))

# read first 5 text lines with utf-8-sig
with open(p, 'r', encoding='utf-8-sig') as f:
    lines = [next(f) for _ in range(5)]
print('\nfirst 5 lines (repr):')
for i, l in enumerate(lines, start=1):
    print(i, repr(l))

# csv.reader header
with open(p, 'r', encoding='utf-8-sig') as f:
    rdr = csv.reader(f)
    header = next(rdr)
print('\ncsv.reader header len:', len(header))
print(header[:12])

# csv.DictReader fieldnames
with open(p, 'r', encoding='utf-8-sig') as f:
    d = csv.DictReader(f)
    print('\nDictReader.fieldnames (first 12):', d.fieldnames[:12] if d.fieldnames else None)
    # show sample first row keys count
    try:
        row = next(d)
        print('sample row keys count:', len(row.keys()))
    except StopIteration:
        print('no data rows')


path -> /Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/OMOP_Schema.csv
raw start repr: b'\xef\xbb\xbfTableName,TableDesc,ColumnName,ColumnDesc,ETL Conventions,ColumnType,Required,IsPK,IsFK,FK table,FK column,FK\r\nPERSON,"This table serves as the central identity management for all Persons in the database. It contains records that uniquely identify each person or patient, and some demographic info'

first 5 lines (repr):
1 'TableName,TableDesc,ColumnName,ColumnDesc,ETL Conventions,ColumnType,Required,IsPK,IsFK,FK table,FK column,FK\n'
2 'PERSON,"This table serves as the central identity management for all Persons in the database. It contains records that uniquely identify each person or patient, and some demographic information.",person_id,It is assumed that every person with a different unique identifier is in fact a different person and should be treated independently.,"Any person linkage that needs to occur to uniquely identify Persons ought to be do

In [57]:
# Reload the implement_code module so edits to read_schema_csv take effect in this kernel
import importlib
import implement_code
importlib.reload(implement_code)
from implement_code import read_schema_csv, normalize_text, encode, evaluate, match_attributes_v2, load_ground_truth
print('Reloaded implement_code and imported helpers')
# quick smoke: call read_schema_csv on OMOP to verify
omop = read_schema_csv('/Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/OMOP_Schema.csv')
print('omop parsed tables:', len(omop), 'first table attrs:', len(omop[0]['attributes']) if omop else 0)


Reloaded implement_code and imported helpers
read_schema_csv: parsed 38 tables and 425 attributes from /Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/OMOP_Schema.csv
omop parsed tables: 38 first table attrs: 19


In [58]:
# --- Embedding: compute and save target vectors for all OMOP attributes (long run)
# Usage: run this cell (it will take several minutes on CPU for ~400 items).
# Set PUSH_TO_PINECONE = True to also attempt an upload (requires PINECONE_API_KEY in env).
import time, pickle, os
from math import ceil

PUSH_TO_PINECONE = False  # set True if you want to upload to Pinecone
OUTPUT_PATH = os.path.join(os.getcwd(), "omop_target_vecs.pkl")

# ensure target_schema is available in the notebook; if not, try to load it
try:
    target_schema
except NameError:
    try:
        from implement_code import read_schema_csv
        target_schema = read_schema_csv(tgt_csv)
        print('Loaded target_schema via implement_code.read_schema_csv')
    except Exception as e:
        raise RuntimeError('target_schema not found in notebook and failed to load: ' + str(e))

# build target_texts + meta (same format used elsewhere in the notebook)
target_texts = []
target_meta = []
for table in target_schema:
    tdesc = table.get('desc', '')
    for attr in table.get('attributes', []):
        full = f"{attr['name']} {attr.get('desc','')} {attr.get('type','')} {tdesc}"
        target_texts.append(full)
        # include desc explicitly so it persists when loading saved metadata
        target_meta.append({'table': table['table'], 'attr': attr['name'], 'name': attr['name'], 'desc': attr.get('desc',''), 'full': full})

print(f"Prepared {len(target_texts)} target texts to embed")

# ensure embeddings object exists (create if missing)
try:
    if 'embeddings' not in globals():
        from langchain.embeddings import HuggingFaceEmbeddings
        embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        print('Created local HuggingFaceEmbeddings instance')
except Exception as e:
    print('Could not (re)create HuggingFaceEmbeddings, will fallback to encode():', e)

# Run embedding (timed)
t0 = time.time()
try:
    target_vecs = embeddings.embed_documents(target_texts)
    print(f"embed_documents completed: produced {len(target_vecs)} vectors")
except Exception as e:
    print('embed_documents failed or raised an error, falling back to encode():', e)
    target_vecs = [encode(t) for t in target_texts]

t1 = time.time()
print('Embedding elapsed time: %.2f s' % (t1 - t0))

# Save vectors + meta to disk for reuse
try:
    with open(OUTPUT_PATH, 'wb') as f:
        pickle.dump({'target_texts': target_texts, 'target_meta': target_meta, 'target_vecs': target_vecs}, f)
    print('Saved vectors+meta to', OUTPUT_PATH)
except Exception as e:
    print('Failed to save vectors to disk:', e)

# Optional: upload to Pinecone (batch upsert)
if PUSH_TO_PINECONE:
    try:
        import pinecone
        api_key = os.environ.get('PINECONE_API_KEY')
        if not api_key:
            raise RuntimeError('PINECONE_API_KEY not found in environment')
        pinecone.init(api_key=api_key)
        index_name = 'omop-attributes'
        dim = len(target_vecs[0]) if target_vecs else 384
        if index_name not in pinecone.list_indexes():
            print('Creating Pinecone index', index_name)
            pinecone.create_index(index_name, dimension=dim, metric='cosine')
        idx = pinecone.Index(index_name)
        batch_size = 100
        total = len(target_vecs)
        for i in range(0, total, batch_size):
            batch_vectors = []
            for j in range(i, min(i + batch_size, total)):
                # id: use numeric stable id; metadata includes table+attr for inspection
                vid = str(j)
                vec = target_vecs[j]
                meta = target_meta[j]
                batch_vectors.append((vid, vec, meta))
            idx.upsert(vectors=batch_vectors)
            print(f"Upserted vectors {i+1}..{min(i+batch_size,total)} (batch size {len(batch_vectors)})")
        print('Finished Pinecone upload to index', index_name)
    except Exception as e:
        print('Pinecone upload failed:', e)

# Quick smoke: print first saved item summary
print('Sample target_meta[0]:', target_meta[0] if target_meta else None)
print('Done. To re-run RAG, load the saved file and set target_vecs,target_meta,target_texts from it.')


RuntimeError: target_schema not found in notebook and failed to load: name 'tgt_csv' is not defined

In [65]:
# --- Hyperparameter sweep (embedding-only) to pick good defaults for RAG
# This cell runs a fast deterministic mapping: for each source attribute, choose the top-1
# target by embedding similarity and accept it when similarity >= threshold.
# It sweeps thresholds and retrieval sizes and picks the best F1 against GT (used only for tuning).

import time, pickle, os

# Load precomputed target vectors if available
DATA_PATH = os.path.join(os.getcwd(), 'omop_target_vecs.pkl')
loaded = False
if os.path.exists(DATA_PATH):
    with open(DATA_PATH, 'rb') as f:
        dd = pickle.load(f)
    target_texts = dd['target_texts']
    target_meta = dd['target_meta']
    target_vecs = dd['target_vecs']
    loaded = True
    print('Loaded precomputed target vectors from', DATA_PATH)
else:
    print('Precomputed vectors not found; computing embeddings (this may take a few minutes)')
    # rebuild target_texts/target_meta from target_schema
    target_texts = []
    target_meta = []
    for table in target_schema:
        tdesc = table.get('desc','')
        for attr in table.get('attributes',[]):
            full = f"{attr['name']} {attr.get('desc','')} {attr.get('type','')} {tdesc}"
            target_texts.append(full)
            target_meta.append({'table': table['table'], 'attr': attr['name'], 'name': attr['name'], 'desc': attr.get('desc',''), 'full': full})
    try:
        target_vecs = embeddings.embed_documents(target_texts)
    except Exception:
        target_vecs = [encode(t) for t in target_texts]
    # save for future
    with open(DATA_PATH, 'wb') as f:
        pickle.dump({'target_texts': target_texts, 'target_meta': target_meta, 'target_vecs': target_vecs}, f)
    print('Saved embeddings to', DATA_PATH)

# load ground truth pairs
gt_pairs = load_ground_truth(gt_csv)
gt_norm_pairs = [(normalize_text(s), normalize_text(t)) for s,t in gt_pairs]

# helper
def deterministic_map(retrieval_k, threshold):
    preds = {}
    for table_s in source_schema:
        sdesc = table_s.get('desc','')
        for attr_s in table_s.get('attributes',[]):
            src = attr_s['name']
            src_full = f"{attr_s['name']} {attr_s.get('desc','')} {attr_s.get('type','')} {sdesc}"
            try:
                q = embeddings.embed_query(src_full)
            except Exception:
                q = encode(src_full)
            sims = [1 - cosine(q, v) for v in target_vecs]
            idxs = sorted(range(len(sims)), key=lambda i: sims[i], reverse=True)[:retrieval_k]
            best_idx = idxs[0]
            best_sim = sims[best_idx]
            if best_sim >= threshold:
                preds[src] = target_meta[best_idx]['attr']
            else:
                preds[src] = None
    eval_res = evaluate({normalize_text(k): (normalize_text(v) if v else None) for k,v in preds.items()}, set(gt_norm_pairs))
    return eval_res, preds

# grid
retrieval_grid = [10, 20, 50, 100]
threshold_grid = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85]
best = None
best_cfg = None
start = time.time()
for rk in retrieval_grid:
    for thr in threshold_grid:
        res, preds = deterministic_map(rk, thr)
        f1 = res['f1']
        print(f"rk={rk}, thr={thr} -> F1={f1:.3f}, P={res['precision']:.3f}, R={res['recall']:.3f}")
        if best is None or f1 > best:
            best = f1
            best_cfg = (rk, thr, res)
end = time.time()
print('\nBest deterministic config:', best_cfg[0], 'retrieval_k, threshold=', best_cfg[1], 'F1=', best)

# set globals so the RAG cell will pick up the tuned values
RETRIEVAL_K = best_cfg[0]
TOP_N = min(50, RETRIEVAL_K)
EMBEDDING_ACCEPT_THRESHOLD = best_cfg[1]
FUZZY_THRESHOLD = globals().get('FUZZY_THRESHOLD', 0.2)
THRESHOLD_SCORE = globals().get('THRESHOLD_SCORE', 0.1)
print('Set globals: RETRIEVAL_K, TOP_N, EMBEDDING_ACCEPT_THRESHOLD')

# show deterministic mapping for best config
res, preds = deterministic_map(RETRIEVAL_K, EMBEDDING_ACCEPT_THRESHOLD)
print('\nDeterministic mapping evaluation with best config:')
print(f"  Ground-truth pairs: {len(gt_norm_pairs)}")
print(f"  Predicted pairs: {len(res['pred_pairs'])}")
print(f"  True positives: {len(res['tp'])}")
print(f"  Precision: {res['precision']:.3f}")
print(f"  Recall:    {res['recall']:.3f}")
print(f"  F1:        {res['f1']:.3f}")

print('\nTip: re-run the full RAG cell (the large one above) now — it will pick up the tuned globals and include descriptions in prompts.')


Loaded precomputed target vectors from /Users/parthawgoswami/Documents/ECHO_Cases/RAG_based_schema_matching/MatchMaker/omop_target_vecs.pkl
rk=10, thr=0.6 -> F1=0.071, P=0.333, R=0.040
rk=10, thr=0.65 -> F1=0.074, P=0.500, R=0.040
rk=10, thr=0.7 -> F1=0.077, P=1.000, R=0.040
rk=10, thr=0.75 -> F1=0.000, P=0.000, R=0.000
rk=10, thr=0.8 -> F1=0.000, P=0.000, R=0.000
rk=10, thr=0.85 -> F1=0.000, P=0.000, R=0.000
rk=20, thr=0.6 -> F1=0.071, P=0.333, R=0.040
rk=20, thr=0.65 -> F1=0.074, P=0.500, R=0.040
rk=20, thr=0.7 -> F1=0.077, P=1.000, R=0.040
rk=20, thr=0.75 -> F1=0.000, P=0.000, R=0.000
rk=20, thr=0.8 -> F1=0.000, P=0.000, R=0.000
rk=20, thr=0.85 -> F1=0.000, P=0.000, R=0.000
rk=50, thr=0.6 -> F1=0.071, P=0.333, R=0.040
rk=50, thr=0.65 -> F1=0.074, P=0.500, R=0.040
rk=50, thr=0.7 -> F1=0.077, P=1.000, R=0.040
rk=50, thr=0.75 -> F1=0.000, P=0.000, R=0.000
rk=50, thr=0.8 -> F1=0.000, P=0.000, R=0.000
rk=50, thr=0.85 -> F1=0.000, P=0.000, R=0.000
rk=100, thr=0.6 -> F1=0.071, P=0.333, R=0